In [1]:
import or_gym
import numpy as np
import matplotlib.pyplot as plt
import copy
from pyomo.environ import *
from pyomo.opt import SolverFactory
import time
%matplotlib inline
plt.style.use('cmu_paper')

In [4]:
env = or_gym.make('VMPacking-v0')
env.step_limit = 5
env.n_pms = 10
%timeit
env.reset()
# env.demand = fixed_demand.copy()
m = ConcreteModel()

m.n = Set(initialize=[i for i in range(env.n_pms)]) # Num Physical Machines
m.v = Set(initialize=[i for i in range(env.step_limit)]) # Num Virtual Machines
m.t = Set(initialize=[i for i in range(env.step_limit)])
m.cpu_demand = Param(m.t,
    initialize={i: j[1] for i, j in enumerate(env.demand)})
m.mem_demand = Param(m.t,
    initialize={i: j[2] for i, j in enumerate(env.demand)})
m.durations = Param(m.v,
    initialize={i: env.step_limit - i for i in range(env.step_limit)})
min_demand = min(m.mem_demand[t] for t in m.t)
m.cpu_limit = env.cpu_capacity
m.mem_limit = env.mem_capacity

m.x = Var(m.n, m.v, m.t, within=Binary) # Assign VM's to machines
m.y = Var(m.n, m.v, within=Binary)
m.z = Var(m.n, m.t, within=Binary) # Number of Machine

@m.Constraint(m.n, m.t)
def cpu_constraint(m, n, t):
    return sum(m.x[n, v, t] * m.cpu_demand[v] 
               for v in m.v) - m.cpu_limit <= 0

@m.Constraint(m.n, m.t)
def mem_constraint(m, n, t):
    return sum(m.x[n, v, t] * m.mem_demand[v]
               for v in m.v) - m.mem_limit <= 0

@m.Constraint(m.n, m.v)
def duration_constraint(m, n, v):
    return sum(m.x[n, v, t] 
            for t in m.t)- m.y[n, v] * m.durations[v] == 0

@m.Constraint(m.v)
def vm_machine_assignment(m, v):
    return sum(m.y[n, v] for n in m.n) == 1

@m.Constraint(m.v, m.t)
def assignment_constraint(m, v, t):
    if t >= v:
        return sum(m.x[n, v, t] for n in m.n) <= 1
    else:
        return sum(m.x[n, v, t] for n in m.n) == 0
    
@m.Constraint(m.n, m.v, m.t)
def time_constraint(m, n, v, t):
    return (m.z[n, t] - m.x[n, v, t]) >= 0
    
# Maximize PM Packing
# m.obj = Objective(expr=(
#     sum(m.y[n, v] for v in m.v for n in m.n)),
#     sense=minimize)
# m.obj = Objective(expr=(
#     sum(m.x[n, v, t]
#         for n in m.n
#         for v in m.v
#         for t in m.t)),
#     sense=minimize)
m.obj = Objective(expr=(
    sum(sum(m.x[n, v, t] * (m.cpu_demand[v] + m.mem_demand[v])
        for v in m.v) - 2 * m.z[n, t]
        for n in m.n
        for t in m.t)),
    sense=maximize)

solver = SolverFactory('gurobi')
solver.options['MIPGap'] = 0.05
solver.options['Threads'] = 4
# solver.options
results = solver.solve(m, tee=True)

plan = []
for v in m.v:
    for t in m.t:
        if v == t:
            for n in m.n:
                if m.x[n, v, t].value is None:
                    continue
                if m.x[n, v, t].value > 0:
                    plan.append([n, v, t])

plan = np.vstack(plan)
print(plan)
rewards = []
actions = []
states = []
done = False
for a in plan:
    t = a[1]
    action = a[0]
    state, r, done, _ = env.step(action)
    states.append(copy.deepcopy(state))
    actions.append(action)
    rewards.append(r)
    if done:
        print(sum(rewards), len(rewards), t)
        break
        
print(m.obj.expr())

Read LP format model from file /tmp/tmpl2xsw873.pyomo.lp
Reading time = 0.00 seconds
x351: 431 rows, 351 columns, 1601 nonzeros
Changed value of parameter MIPGap to 0.05
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Changed value of parameter Threads to 4
   Prev: 0  Min: 0  Max: 1024  Default: 0
Optimize a model with 431 rows, 351 columns and 1601 nonzeros
Variable types: 1 continuous, 350 integer (350 binary)
Coefficient statistics:
  Matrix range     [3e-02, 5e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -18.1088154
Presolve removed 286 rows and 261 columns
Presolve time: 0.00s
Presolved: 145 rows, 90 columns, 330 nonzeros
Variable types: 0 continuous, 90 integer (90 binary)

Root relaxation: objective -6.108815e+00, 5 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap

In [90]:
def build_online_vm_opt(env):
    m = ConcreteModel()
    
    m.t0 = env.current_step # Current time
    m.n = Set(initialize=[i for i in range(env.n_pms)]) # Num Physical Machines
    # One new VM at each time step
    m.v = Set(initialize=[v for v in range(env.current_step+1)])
    m.t = Set(initialize=[t for t in range(env.step_limit)])
    
    m.cpu_demand = Param(m.v,
        initialize={i: j[1] for i, j in enumerate(env.demand[:m.t0+1])})
    m.mem_demand = Param(m.v,
        initialize={i: j[2] for i, j in enumerate(env.demand[:m.t0+1])})
    m.durations = Param(m.v,
        initialize={v: env.step_limit - v for v in m.v})
    m.cpu_limit = env.cpu_capacity
    m.mem_limit = env.mem_capacity

    m.x = Var(m.n, m.v, m.t, within=Binary) # Assign VM's to machines at t
    m.y = Var(m.n, m.v, within=Binary) # Assign VM's to machine
    m.z = Var(m.n, m.t, within=Binary) # Time when machine is occupied
    
    # Fix variables for pre-existing values
    if m.t0 > 0:
        for k in env.assignment: # Times
            n = env.assignment[k]
            m.y[n, k].fix(1)
            for t in m.t:
                m.x[n, k, t].fix(1)
        
    @m.Constraint(m.n, m.t)
    def cpu_constraint(m, n, t):
        return sum(m.x[n, v, t] * m.cpu_demand[v] 
                   for v in m.v) - m.cpu_limit <= 0

    @m.Constraint(m.n, m.t)
    def mem_constraint(m, n, t):
        return sum(m.x[n, v, t] * m.mem_demand[v]
                   for v in m.v) - m.mem_limit <= 0

    @m.Constraint(m.n, m.v)
    def duration_constraint(m, n, v):
        return sum(m.x[n, v, t] 
                for t in m.t)- m.y[n, v] * m.durations[v] == 0

    @m.Constraint(m.v)
    def vm_machine_assignment(m, v):
        return sum(m.y[n, v] for n in m.n) == 1

    @m.Constraint(m.v, m.t)
    def assignment_constraint(m, v, t):
        if t >= v:
            return sum(m.x[n, v, t] for n in m.n) <= 1
        else:
            return sum(m.x[n, v, t] for n in m.n) == 0

    @m.Constraint(m.n, m.v, m.t)
    def time_constraint(m, n, v, t):
        return (m.z[n, t] - m.x[n, v, t]) >= 0

    # Maximize PM Packing
    m.obj = Objective(expr=(
        sum(sum(m.x[n, v, t] * (m.cpu_demand[v] + m.mem_demand[v])
            for v in m.v) - 2 * m.z[n, t]
            for n in m.n
            for t in m.t)),
        sense=maximize)
    return m

In [44]:
solver = SolverFactory('gurobi')
env.reset()
m = build_online_vm_opt(env)
results = solver.solve(m)
action = extract_plan(m)[0]
out = env.step(action)

In [46]:
m = build_online_vm_opt(env)
results = solver.solve(m)
action = extract_plan(m)[0]
out = env.step(action)

    model=unknown;
        message from solver=Problem proven to be infeasible or unbounded.


In [64]:
print('x vars')
_ = [print("N={}\tV={}\tt={}\tx={}".format(idx[0], idx[1], idx[2], m.x[idx].value))
     for idx in m.x if m.x[idx].value is not None]
print('\ny vars')
_ = [print("N={}\tV={}\ty={}".format(idx[0], idx[1], m.y[idx].value)) 
     for idx in m.y if m.y[idx].value is not None]
print('\nz vars')
_ = [print("N={}\tt={}\tz={}".format(idx[0], idx[1], m.z[idx].value)) 
     for idx in m.z if m.z[idx].value is not None]

x vars
N=0	V=0	t=1	x=1
N=0	V=0	t=2	x=1
N=0	V=0	t=3	x=1
N=0	V=0	t=4	x=1

y vars
N=0	V=0	y=1

z vars


In [65]:
if m.t0 > 0:
    for k in env.assignment: # Times
        n = env.assignment[k]
        for t in m.t:
            m.x[n, k, t].fix(1)
        m.y[n, k].fix(1)

In [56]:
[m.cpu_demand[idx] for idx in m.cpu_demand]

[0.15515643529097686, 0.16309118762937813]

In [58]:
[m.mem_demand[idx] for idx in m.mem_demand]

[0.11428571, 0.45714286]

In [73]:
env.reset()
_ = env.step(0)

In [87]:
env.assignment

{0: 0}

In [86]:
m = ConcreteModel()
    
m.t0 = env.current_step # Current time
m.n = Set(initialize=[i for i in range(env.n_pms)]) # Num Physical Machines
# One new VM at each time step
m.v = Set(initialize=[v for v in range(m.t0+1)])
m.t = Set(initialize=[t for t in range(m.t0, env.step_limit)])

m.cpu_demand = Param(m.v,
    initialize={i: j[1] for i, j in enumerate(env.demand[:m.t0+1])})
m.mem_demand = Param(m.v,
    initialize={i: j[2] for i, j in enumerate(env.demand[:m.t0+1])})
m.durations = Param(m.v,
    initialize={v: env.step_limit - v for v in m.v})
m.cpu_limit = env.cpu_capacity
m.mem_limit = env.mem_capacity

m.x = Var(m.n, m.v, m.t, within=Binary) # Assign VM's to machines at t
m.y = Var(m.n, m.v, within=Binary) # Assign VM's to machine
m.z = Var(m.n, m.t, within=Binary) # Time when machine is occupied

# Fix variables for pre-existing values
if m.t0 > 0:
    for k in env.assignment:
        n = env.assignment[k]
        m.y[n, k].fix(1)
        for t in m.t:
            if t >= k:
                m.x[n, k, t].fix(1)
            else:
                m.x[n, k, t].fix(0)

@m.Constraint(m.n, m.t)
def cpu_constraint(m, n, t):
    return sum(m.x[n, v, t] * m.cpu_demand[v] 
               for v in m.v) - m.cpu_limit <= 0

@m.Constraint(m.n, m.t)
def mem_constraint(m, n, t):
    return sum(m.x[n, v, t] * m.mem_demand[v]
               for v in m.v) - m.mem_limit <= 0

@m.Constraint(m.n, m.v)
def duration_constraint(m, n, v):
    return sum(m.x[n, v, t] 
            for t in m.t)- m.y[n, v] * m.durations[v] == 0

@m.Constraint(m.v)
def vm_machine_assignment(m, v):
    return sum(m.y[n, v] for n in m.n) == 1

@m.Constraint(m.v, m.t)
def assignment_constraint(m, v, t):
    if t >= v:
        return sum(m.x[n, v, t] for n in m.n) <= 1
    else:
        return sum(m.x[n, v, t] for n in m.n) == 0

@m.Constraint(m.n, m.v, m.t)
def time_constraint(m, n, v, t):
    return (m.z[n, t] - m.x[n, v, t]) >= 0

# Maximize PM Packing
m.obj = Objective(expr=(
    sum(sum(m.x[n, v, t] * (m.cpu_demand[v] + m.mem_demand[v])
        for v in m.v) - 2 * m.z[n, t]
        for n in m.n
        for t in m.t)),
    sense=maximize)

results = solver.solve(m)

    model=unknown;
        message from solver=Problem proven to be infeasible or unbounded.


In [84]:
env.assignment

{0: 0}

In [80]:
[(i, j) for i, j in enumerate(env.demand[m.t0:m.t0+1], m.t0)]

[(1, array([0.        , 0.14914529, 0.45714286]))]

In [91]:
env = or_gym.make('VMPacking-v0')
env.n_pms = 10
env.step_limit = 5
solver = SolverFactory('gurobi')
env.reset()

done = False
actions = []
rewards = []
times, obj_vals, bin_vars = [], [], []
count = 0
t0 = time.time()
while done==False:
    t1 = time.time()
    m = build_online_vm_opt(env)
    results = solver.solve(m)
    # Get action plan
    plan = extract_plan(m)
    action = plan[-1] #.take(0)
    s, r, done, _ = env.step(action)
    actions.append(action)
    rewards.append(r)
    t2 = time.time()
    count += 1
    times.append(t2-t1)
    obj_vals.append(m.obj.expr())
    bin_vars.append(results['Problem'][0]['Number of binary variables'])
    if count % 10 == 0:
        print("Ep: {}\tSteps/s: {:.2f}\tTotal time (s): {:.2f}\tProjected Reward: {:.2f}".format(
            count, 10/np.mean(times[-10:]), t2-t0, obj_vals[-1]))
        
print("Total Time (s):\t\t{:.2f}\t".format(t2-t0))
print("Mean Binary Variables:\t{:.0f}".format(np.mean(bin_vars)))

    model=unknown;
        message from solver=Problem proven to be infeasible or unbounded.
ERROR: evaluating object as numeric value: x[0,2,0]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object x[0,2,0]


ValueError: No value for uninitialized NumericValue object x[0,2,0]

In [10]:
# Get plan
def extract_vm_packing_plan(model):
    plan = []
    for v in m.v:
        for t in m.t:
            if v == t:
                for n in m.n:
                    if m.x[n, v, t].value is None:
                        continue
                    if m.x[n, v, t].value > 0:
                        plan.append(n)

    return np.vstack(plan)

In [34]:
def extract_plan(model):
    plan = []
    for n in m.n:
        for v in m.v:
            if m.x[n, v, m.t0].value is None:
                continue
            elif m.x[n, v, m.t0].value > 0:
                plan.append(n)
    return plan

ERROR: evaluating object as numeric value: x[0,1,1]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object x[0,1,1]


ValueError: No value for uninitialized NumericValue object x[0,1,1]